# Extraction and Retrieval with Docling and Langchain DocLoaders

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
from openai import OpenAI
from qdrant_client import QdrantClient
from IPython.display import display, Markdown


c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Params

In [3]:
INDEX = "althera-openai-01"
EMB_MODEL = "text-embedding-3-small"
EMB_DIM = 1536
RETRIEVE_K=5

# Environment Variables

In [4]:
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


# Clients

In [5]:

client_qdrant = QdrantClient(":memory:")

client_openai = OpenAI(
    api_key=OPENAI_API_KEY
)


In [6]:
path_data = Path() / "data"
path_input = path_data / "raw"

# Extraction

In [7]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType


FILE_PATH = [(path_input / "Divulgacion-Planetaria-Althera.pdf").as_posix()]  # for multiple files
EXPORT_TYPE = ExportType.MARKDOWN   # ExportType.DOC_CHUNKS
TOKENIZER_NAME ="cl100k_base"

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
)

corpus_docling = loader.load()

c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)
c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\.venv\Lib\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


In [8]:
type(corpus_docling)

list

In [9]:
# ExportType.DOC_CHUNKS -> 68 chunks
# ExportType.MARKDOWN -> 1 chunk
len(corpus_docling)

1

In [10]:
type(corpus_docling[0])

langchain_core.documents.base.Document

In [11]:
Markdown(corpus_docling[0].page_content[:500])

## Un nuevo y fascinante vecino: Althéra

## Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

## 1. Historia del descubrimiento

## 1.1 Primeras observaciones y sospechas iniciales

El sistema binario Althéra ( HD 4579 AB ) fue detectado por primera vez en el 

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter_rcs = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name=TOKENIZER_NAME,
    chunk_size=256,  # from_tiktoken_encoder: tokens
    chunk_overlap=25  # tokens
)
corpus_rcs = text_splitter_rcs.split_documents(corpus_docling)  # corpus is a list of LG Documents
len(corpus_rcs)


51

# Index

In [13]:
from uuid import uuid4

lst_chunks = [doc.page_content for doc in corpus_rcs]

In [14]:
lst_embeddings = []

response = client_openai.embeddings.create(
    input=lst_chunks,
    model=EMB_MODEL
)

for vec in response.data:
    lst_embeddings.append(vec.embedding)


print(f"{len(lst_embeddings)}")
print(f"{len(lst_embeddings[0])}")  # embeddings dimension

51
1536


In [15]:
from qdrant_client import models, QdrantClient

In [16]:
# src.embeddings. create_index_points() 
lst_qdrant_pts = []

for content, vector in zip(lst_chunks, lst_embeddings):
    qdrant_point = models.PointStruct(
        id=str(uuid4()),  # uuid aleatorio
        payload={
            "text": content,
            "source": "Divulgacion-Planetaria-Althera.pdf",
        },
        vector=vector  # se puede buscar directamente en la lista por indice del DF pivotado
    )
    lst_qdrant_pts.append(qdrant_point)

len(lst_qdrant_pts) == len(corpus_rcs)  # comprobar

True

In [23]:
type(client_qdrant.get_collection(INDEX).points_count)

int

In [24]:
from qdrant_client.http.models import Distance, VectorParams

if not client_qdrant.collection_exists(INDEX):
    print(f"Creating collection {INDEX}...")
    client_qdrant.create_collection(
        collection_name=INDEX,
        vectors_config=VectorParams(size=EMB_DIM, distance=Distance.COSINE),
    )
else:
    print(f"Collection {INDEX} already exists")

if client_qdrant.get_collection(INDEX).points_count ==0:
    print("Loading chunks on VDB...")
    client_qdrant.upsert(
        collection_name=INDEX,
        points=lst_qdrant_pts
    )
else:
    print("There are already some chunks on the VDB")



Collection althera-openai-01 already exists
Loading chunks on VDB...


# Retrieval

In [25]:
# Embbed the query
_q_text = "A que distancia se encuentra del sistema"


response = client_openai.embeddings.create(
    input=[_q_text],
    model=EMB_MODEL
)
_q_vec = response.data[0].embedding


# Query 
resp = client_qdrant.query_points(
    collection_name=INDEX,
    query=_q_vec,  
    limit=3  # K 
)

# idx: row in dataset
# j: position in text_columns (relevancy)
print(f"Question: {_q_text}")
for point in resp.points:
    doc_retrieved = point.payload['text']
    print(f"{point.id=}")
    print(f"{point.score=}")
    print(f"Doc: {doc_retrieved[:500]}...")
    print("-"*30)

Question: A que distancia se encuentra del sistema
point.id='8fc2045e-886f-41f5-9513-ed62e952b74f'
point.score=0.49210520058439167
Doc: ## 3.4 Interacción gravitatoria y efectos sobre la zona habitable

La órbita mutua de las dos estrellas, con una separación media de 0,42 UA y una excentricidad de 0,12, genera un entorno gravitacional complejo. Sin embargo, simulaciones dinámicas realizadas por el Centro de Dinámica Planetaria de París muestran que:

- Las órbitas de los planetas a más de 1,8 UA del centro binario son estables a escalas de miles de millones de años.
- La zona habitable circumbinaria se extiende aproximadamente ...
------------------------------
point.id='2290c012-091a-4d55-8427-825eadd48699'
point.score=0.48706394700963884
Doc: ## 2. Conoce a Althéra

## 2.1 Localización en la galaxia

El sistema binario Althéra ( HD 4579 AB ) se encuentra en el brazo de Orión de la Vía Láctea , a una distancia de 42,7 años luz de la Tierra. Su posición aparente en el cielo se localiz